# https://jaykmody.com/blog/gpt-from-scratch

In [1]:
from utils import load_encoder_hparams_and_params
encoder, hparams, params = load_encoder_hparams_and_params("124M", "models")

2023-02-12 18:58:52.783854: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-12 18:58:52.896682: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# BPE tokenizer used by GPT-2

ids = encoder.encode("Not all heroes wear capes.")
ids

[3673, 477, 10281, 5806, 1451, 274, 13]

In [3]:
[encoder.decoder[i] for i in ids]

['Not', 'Ġall', 'Ġheroes', 'Ġwear', 'Ġcap', 'es', '.']

In [4]:
hparams

{'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 768, 'n_head': 12, 'n_layer': 12}

In [5]:
import numpy as np

def shape_tree(d):
    if isinstance(d, np.ndarray):
        return list(d.shape)
    elif isinstance(d, list):
        return [shape_tree(v) for v in d]
    elif isinstance(d, dict):
        return {k: shape_tree(v) for k, v in d.items()}
    else:
        ValueError("uh oh")

print(shape_tree(params))

# shapes replced with the hparam they correspond to
# {
#     "wpe": [n_ctx, n_embd],
#     "wte": [n_vocab, n_embd],    
#     "ln_f": {"b": [n_embd], "g": [n_embd]},
#     "blocks": [
#         {
#             "attn": {
#                 "c_attn": {"b": [3*n_embd], "w": [n_embd, 3*n_embd]},
#                 "c_proj": {"b": [n_embd], "w": [n_embd, n_embd]},
#             },
#             "ln_1": {"b": [n_embd], "g": [n_embd]},
#             "ln_2": {"b": [n_embd], "g": [n_embd]},
#             "mlp": {
#                 "c_fc": {"b": [4*n_embd], "w": [n_embd, 4*n_embd]},
#                 "c_proj": {"b": [n_embd], "w": [4*n_embd, n_embd]},
#             },
#         },
#         ... # repeat for n_layers
#     ]
# }

{'blocks': [{'attn': {'c_attn': {'b': [2304], 'w': [768, 2304]}, 'c_proj': {'b': [768], 'w': [768, 768]}}, 'ln_1': {'b': [768], 'g': [768]}, 'ln_2': {'b': [768], 'g': [768]}, 'mlp': {'c_fc': {'b': [3072], 'w': [768, 3072]}, 'c_proj': {'b': [768], 'w': [3072, 768]}}}, {'attn': {'c_attn': {'b': [2304], 'w': [768, 2304]}, 'c_proj': {'b': [768], 'w': [768, 768]}}, 'ln_1': {'b': [768], 'g': [768]}, 'ln_2': {'b': [768], 'g': [768]}, 'mlp': {'c_fc': {'b': [3072], 'w': [768, 3072]}, 'c_proj': {'b': [768], 'w': [3072, 768]}}}, {'attn': {'c_attn': {'b': [2304], 'w': [768, 2304]}, 'c_proj': {'b': [768], 'w': [768, 768]}}, 'ln_1': {'b': [768], 'g': [768]}, 'ln_2': {'b': [768], 'g': [768]}, 'mlp': {'c_fc': {'b': [3072], 'w': [768, 3072]}, 'c_proj': {'b': [768], 'w': [3072, 768]}}}, {'attn': {'c_attn': {'b': [2304], 'w': [768, 2304]}, 'c_proj': {'b': [768], 'w': [768, 768]}}, 'ln_1': {'b': [768], 'g': [768]}, 'ln_2': {'b': [768], 'g': [768]}, 'mlp': {'c_fc': {'b': [3072], 'w': [768, 3072]}, 'c_proj'

In [6]:
import tensorflow as tf
tf_ckpt_path = tf.train.latest_checkpoint("models/124M")
for name, _ in tf.train.list_variables(tf_ckpt_path):
    arr = tf.train.load_variable(tf_ckpt_path, name).squeeze()
    print(f"{name}: {arr.shape}")

model/h0/attn/c_attn/b: (2304,)
model/h0/attn/c_attn/w: (768, 2304)
model/h0/attn/c_proj/b: (768,)
model/h0/attn/c_proj/w: (768, 768)
model/h0/ln_1/b: (768,)
model/h0/ln_1/g: (768,)
model/h0/ln_2/b: (768,)
model/h0/ln_2/g: (768,)
model/h0/mlp/c_fc/b: (3072,)
model/h0/mlp/c_fc/w: (768, 3072)
model/h0/mlp/c_proj/b: (768,)
model/h0/mlp/c_proj/w: (3072, 768)
model/h1/attn/c_attn/b: (2304,)
model/h1/attn/c_attn/w: (768, 2304)
model/h1/attn/c_proj/b: (768,)
model/h1/attn/c_proj/w: (768, 768)
model/h1/ln_1/b: (768,)
model/h1/ln_1/g: (768,)
model/h1/ln_2/b: (768,)
model/h1/ln_2/g: (768,)
model/h1/mlp/c_fc/b: (3072,)
model/h1/mlp/c_fc/w: (768, 3072)
model/h1/mlp/c_proj/b: (768,)
model/h1/mlp/c_proj/w: (3072, 768)
model/h10/attn/c_attn/b: (2304,)
model/h10/attn/c_attn/w: (768, 2304)
model/h10/attn/c_proj/b: (768,)
model/h10/attn/c_proj/w: (768, 768)
model/h10/ln_1/b: (768,)
model/h10/ln_1/g: (768,)
model/h10/ln_2/b: (768,)
model/h10/ln_2/g: (768,)
model/h10/mlp/c_fc/b: (3072,)
model/h10/mlp/c_fc

In [7]:
## GELU (Gaussian Error Linear Unit) activation function
def gelu(x):
    return 0.5 * x * (1 + np.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * x**3)))
gelu(np.array([[1, 2], [-4, 0]]))

array([[ 8.41191991e-01,  1.95459769e+00],
       [-7.02459482e-05,  0.00000000e+00]])

In [8]:
## Softmax
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)
x = softmax(np.array([[2, 100], [-5, 0]]))
x, x.sum(axis=-1)

(array([[2.74878501e-43, 1.00000000e+00],
        [6.69285092e-03, 9.93307149e-01]]),
 array([1., 1.]))

In [9]:
## Layer normalization
def layer_norm(x, g, b, eps: float = 1e-5):
    mean = np.mean(x, axis=-1, keepdims=True)
    variance = np.var(x, axis=-1, keepdims=True)
    x = (x - mean) / np.sqrt(variance + eps)  # normalize x to have mean=0 and var=1 over last axis
    return g * x + b  # scale and offset with gamma/beta params
x = np.array([[2, 2, 3], [-5, 0, 1]])
x = layer_norm(x, g=np.ones(x.shape[-1]), b=np.zeros(x.shape[-1]))
x, x.var(axis=-1), x.mean(axis=-1)


(array([[-0.70709087, -0.70709087,  1.41418174],
        [-1.39700038,  0.50800014,  0.88900024]]),
 array([0.999955  , 0.99999855]),
 array([-2.96059473e-16, -3.70074342e-17]))

In [10]:
## Linear layer
def linear(x, w, b):  # [m, in], [in, out], [out] -> [m, out]
    return x @ w + b
x = np.random.normal(size=(64, 784)) # input dim = 784, batch/sequence dim = 64
w = np.random.normal(size=(784, 10)) # output dim = 10
b = np.random.normal(size=(10,))
x.shape, linear(x, w, b).shape # shapes before / after linear projection

((64, 784), (64, 10))

In [11]:
# Example inference
import gpt2
gpt2.main(prompt="Alan Turing theorized that computers would one day become", n_tokens_to_generate=8)

generating: 100%|██████████| 8/8 [00:11<00:00,  1.43s/it]


' the most powerful machines on the planet.'